# Find KEGG associations

This notebook will create a table that has the KEGG pathways that are associated with all genes, but we are particularly interested in those that are associated with the most and least stable genes.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import random
import pandas as pd
from scripts import paths, utils, modules, annotations

random.seed(1)

In [2]:
# Output files
pao1_out_filename = "pao1_core_similarity_associations_spell.tsv"
pa14_out_filename = "pa14_core_similarity_associations_spell.tsv"

In [3]:
# Load transcriptional similarity df
pao1_similarity_scores_filename = "pao1_core_similarity_expression_stats_spell.tsv"
pa14_similarity_scores_filename = "pa14_core_similarity_expression_stats_spell.tsv"

pao1_similarity_scores = pd.read_csv(
    pao1_similarity_scores_filename, sep="\t", header=0, index_col=0
)
pa14_similarity_scores = pd.read_csv(
    pa14_similarity_scores_filename, sep="\t", header=0, index_col=0
)

In [4]:
print(pao1_similarity_scores.shape)
pao1_similarity_scores.head()

(5349, 14)


,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label,mean expression,standard deviation expression,min expression,25% expression,50% expression,75% expression,max expression,variance expression,range expression
PAO1 id,,,,,,,,,,,,,,
PA1839,PA14_40730,0.479379,1.533344e-305,NaN,NaN,132.804962,87.734616,4.087634,73.908945,115.812474,171.017374,821.445691,7.697363e+03,817.358057
PA1093,PA14_50280,0.399669,2.303959e-204,NaN,NaN,999.954597,1367.391295,0.057625,280.483419,722.971734,1317.661680,22264.320777,1.869759e+06,22264.263152
PA1947,PA14_39330,0.384830,2.260974e-188,rbsA,NaN,312.349358,380.589548,0.209768,107.163043,191.967653,381.118755,3768.828667,1.448484e+05,3768.618898
PA4672,PA14_61790,0.386774,2.023694e-190,NaN,NaN,448.826146,933.567419,6.160916,84.240169,160.854155,388.313280,12997.089379,8.715481e+05,12990.928463
PA2251,PA14_35470,0.259699,3.537348e-83,NaN,NaN,109.288623,80.205609,0.062196,64.545999,90.854359,137.165351,652.879274,6.432940e+03,652.817078


In [5]:
print(pa14_similarity_scores.shape)
pa14_similarity_scores.head()

(5348, 14)


,PAO1 homolog id,Transcriptional similarity across strains,P-value,Name,label,mean expression,standard deviation expression,min expression,25% expression,50% expression,75% expression,max expression,variance expression,range expression
PA14 id,,,,,,,,,,,,,,
PA14_01690,PA0138,0.398073,1.444998e-202,NaN,NaN,48.146302,144.110331,2.170836,13.137108,23.539019,48.905936,2527.689060,2.076779e+04,2525.518223
PA14_05540,PA0426,0.283820,1.230401e-99,mexB,NaN,4072.376017,2557.907831,265.981640,2333.523085,3525.336963,5254.590539,18765.637983,6.542892e+06,18499.656344
PA14_19630,PA3438,0.326167,8.899237e-133,folE1,NaN,335.208749,310.824107,13.523229,166.456655,231.032201,347.361538,2252.077923,9.661163e+04,2238.554694
PA14_12730,PA3953,0.299629,2.300539e-111,NaN,NaN,24.353518,18.451461,2.145587,12.714652,20.589063,31.881316,248.122333,3.404564e+02,245.976746
PA14_69780,PA5286,0.468224,1.191668e-289,NaN,NaN,397.297338,490.418270,15.377551,112.325047,232.872707,436.557607,3209.148659,2.405101e+05,3193.771108


In [6]:
# Load KEGG pathway data
pao1_pathway_filename = "https://raw.githubusercontent.com/greenelab/adage/7a4eda39d360b224268921dc1f2c14b32788ab16/Node_interpretation/pseudomonas_KEGG_terms.txt"

In [7]:
pao1_pathways = annotations.load_format_KEGG(pao1_pathway_filename)
pao1_pathways.head()

,1,2
0,,
KEGG-Pathway-pae00072: Synthesis and degradation of ketone bodies,10,"{PA3589, PA1999, PA2003, PA2000, PA2553, PA173..."
KEGG-Pathway-pae00071: Fatty acid degradation,32,"{PA3589, PA1525, PA3013, PA4994, PA2553, PA153..."
KEGG-Pathway-pae00903: Limonene and pinene degradation,9,"{PA4899, PA2475, PA1821, PA1748, PA3426, PA333..."
KEGG-Pathway-pae00380: Tryptophan metabolism,27,"{PA3589, PA4236, PA2080, PA2553, PA4342, PA461..."
KEGG-Pathway-pae00900: Terpenoid backbone biosynthesis,16,"{PA3589, PA3803, PA4043, PA2553, PA1736, PA456..."


## Pathway annotations to PA14

The annotations we have are only for PAO1 genes, so we will map PAO1 core genes to PA14 core genes to add annotations to PA14. This is possible since we are focused on only core genes, which have homologs between PAO1 and PA14

In [8]:
pao1_annotation_filename = paths.GENE_PAO1_ANNOT
gene_mapping_pao1 = utils.get_pao1_pa14_gene_map(pao1_annotation_filename, "pao1")

In [9]:
gene_mapping_pao1 = gene_mapping_pao1["PA14_ID"].to_frame()

## Get pathway associations for all genes

In [10]:
def get_associated_pathways(genes_):
    rows = []
    for gene_id in genes_:
        pathway_bool = [
            gene_id in pao1_pathways.loc[pathway, 2] for pathway in pao1_pathways.index
        ]
        found_pathways = list(pao1_pathways[pathway_bool].index)
        rows.append({"gene id": gene_id, "pathways present": found_pathways})
    return pd.DataFrame(rows).set_index("gene id")

In [11]:
# Get KEGG associations for all genes in PAO1
all_pao1_gene_ids = list(pao1_similarity_scores.index)
pao1_associations = get_associated_pathways(all_pao1_gene_ids)

In [12]:
print(pao1_associations.shape)
pao1_associations.head()

(5349, 1)


,pathways present
gene id,
PA1839,[]
PA1093,[]
PA1947,[]
PA4672,[]
PA2251,[]


In [13]:
# Map PA14 gene ids
pa14_associations = pao1_associations.merge(
    gene_mapping_pao1, left_index=True, right_index=True
)
pa14_associations.set_index("PA14_ID", inplace=True)
print(pa14_associations.shape)
pa14_associations.head()

(5349, 1)


,pathways present
PA14_ID,
PA14_40730,[]
PA14_50280,[]
PA14_39330,[]
PA14_61790,[]
PA14_35470,[]


In [14]:
# Merge KEGG associations with transcriptional similarity information
pao1_associations = pao1_similarity_scores.merge(
    pao1_associations, left_index=True, right_index=True, how="left"
)
pa14_associations = pa14_similarity_scores.merge(
    pa14_associations, left_index=True, right_index=True, how="left"
)

In [15]:
print(pao1_associations.shape)
pao1_associations.head()

(5349, 15)


,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label,mean expression,standard deviation expression,min expression,25% expression,50% expression,75% expression,max expression,variance expression,range expression,pathways present
PAO1 id,,,,,,,,,,,,,,,
PA1839,PA14_40730,0.479379,1.533344e-305,NaN,NaN,132.804962,87.734616,4.087634,73.908945,115.812474,171.017374,821.445691,7.697363e+03,817.358057,[]
PA1093,PA14_50280,0.399669,2.303959e-204,NaN,NaN,999.954597,1367.391295,0.057625,280.483419,722.971734,1317.661680,22264.320777,1.869759e+06,22264.263152,[]
PA1947,PA14_39330,0.384830,2.260974e-188,rbsA,NaN,312.349358,380.589548,0.209768,107.163043,191.967653,381.118755,3768.828667,1.448484e+05,3768.618898,[]
PA4672,PA14_61790,0.386774,2.023694e-190,NaN,NaN,448.826146,933.567419,6.160916,84.240169,160.854155,388.313280,12997.089379,8.715481e+05,12990.928463,[]
PA2251,PA14_35470,0.259699,3.537348e-83,NaN,NaN,109.288623,80.205609,0.062196,64.545999,90.854359,137.165351,652.879274,6.432940e+03,652.817078,[]


In [16]:
print(pa14_associations.shape)
pa14_associations.head()

(5348, 15)


,PAO1 homolog id,Transcriptional similarity across strains,P-value,Name,label,mean expression,standard deviation expression,min expression,25% expression,50% expression,75% expression,max expression,variance expression,range expression,pathways present
PA14 id,,,,,,,,,,,,,,,
PA14_01690,PA0138,0.398073,1.444998e-202,NaN,NaN,48.146302,144.110331,2.170836,13.137108,23.539019,48.905936,2527.689060,2.076779e+04,2525.518223,[]
PA14_05540,PA0426,0.283820,1.230401e-99,mexB,NaN,4072.376017,2557.907831,265.981640,2333.523085,3525.336963,5254.590539,18765.637983,6.542892e+06,18499.656344,[KEGG-Pathway-pae00312: beta-Lactam resistance...
PA14_19630,PA3438,0.326167,8.899237e-133,folE1,NaN,335.208749,310.824107,13.523229,166.456655,231.032201,347.361538,2252.077923,9.661163e+04,2238.554694,"[KEGG-Pathway-pae00790: Folate biosynthesis, K..."
PA14_12730,PA3953,0.299629,2.300539e-111,NaN,NaN,24.353518,18.451461,2.145587,12.714652,20.589063,31.881316,248.122333,3.404564e+02,245.976746,[]
PA14_69780,PA5286,0.468224,1.191668e-289,NaN,NaN,397.297338,490.418270,15.377551,112.325047,232.872707,436.557607,3209.148659,2.405101e+05,3193.771108,[]


In [17]:
# Save
pao1_associations.to_csv(pao1_out_filename, sep="\t")
pa14_associations.to_csv(pa14_out_filename, sep="\t")